# Imports and Settings

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAI

In [2]:
load_dotenv()

True

# Simple Invocations

In [7]:
llm = OpenAI()
chat_model = ChatOpenAI()

In [8]:
from langchain_core.messages import HumanMessage

In [9]:
text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [11]:
print(llm.invoke(text))



"Rainbow Socks Co." or "Chroma Socks Co."


In [12]:
chat_model.invoke(messages)

AIMessage(content='Rainbow Threads Co.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 22, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa65f054-7d29-4274-955d-726cac33c160-0', usage_metadata={'input_tokens': 22, 'output_tokens': 5, 'total_tokens': 27})

In [13]:
chat_model.invoke(messages).content

'Rainbow Footwear Co.'

> **NOTE**
> - The LLM returns a string.
> - The ChatModel returns a `AIMessage` object

# Prompt Templates

Most LLM applications **do not pass user input directly into an LLM**.

Usually, they will **add the user input to a larger piece of text, called a *prompt template**, that **provides additional context on the specific task at hand**.

In the previous example, the text we passed to the model contained instructions to generate a company name. For our application, **it would be great if the user only had to provide the description of a company/product, without worrying about giving the model instructions**.

**`PromptTemplate`s** help with exactly this! They **bundle up all the logic for going from user input into the fully formatted prompt**.

This can start off very simple, for example:

In [14]:
from langchain_core.prompts import PromptTemplate

In [15]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")

In [16]:
type(prompt)

langchain_core.prompts.prompt.PromptTemplate

In [17]:
prompt

PromptTemplate(input_variables=['product'], template='What is a good name for a company that makes {product}?')

> Essentially, that's an object that enhances ***f-strings*** functionalities

In [18]:
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

There are several advantages of using this over raw string formatting:

- You can "partial" out variables, e.g **you can only format some of the variables at a time**.
- You can compose them together, easily combining different templates into a single prompt.

👉 For further details, see the [**section about prompts**](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/)

`PromptTemplate`s can also be used to **produce a list of messages**.

In this case, the prompt not only contains information about the content, but also each message:
- its role,
- its position in the list...

Here, what happens most often is that a `ChatPromptTemplate` is a list of `ChatMessageTemplate`s

Each `ChatMessageTemplate` contains instructions for how to format that `ChatMessage`, its role and then also its content.

In [19]:
from langchain_core.prompts.chat import ChatPromptTemplate

In [25]:
system_template = "You're a helpful assistant that translates {input_language} to {output_language}"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

In [26]:
type(chat_prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [27]:
chat_prompt

ChatPromptTemplate(input_variables=['input_language', 'output_language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template="You're a helpful assistant that translates {input_language} to {output_language}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [28]:
chat_prompt.format_messages(
    input_language="English",
    output_language="French",
    text="I love programming"
)

[SystemMessage(content="You're a helpful assistant that translates English to French"),
 HumanMessage(content='I love programming')]

`ChatPromptTemplate`s can also be constructed in other ways. See the [**section on prompts**](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/) for more details.

# Output Parsers

`OutputParser`s convert the raw output of a language model into a format that can be used downstream.

There are a few main types of `OutputParser`s, including:
- Convert a text from `LLM` into structured information (e.g. JSON).
- Convert a `ChatMessage` into a string.
- Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

For full information on this, see the [**section on output parsers**](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/)

In this getting started guide, we use a simple one that parses a list of comma separated values.

In [29]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [30]:
output_parser = CommaSeparatedListOutputParser()
output_parser.parse("Hi, bye!")

['Hi', 'bye!']

# Composing with LCEL

We can now **combine all these elements into a single chain**.

This chain will:
- take input variables,
- pass those to a prompt template to create a prompt,
- pass the prompt to a language model,
- pass the output to an output parser.

This is a **convenient way to bundle up a modular piece of logic**.

In [33]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = chat_prompt | chat_model | output_parser

> **NOTE**
> 
> Passing `format_instructions=output_parser.get_format_instructions` let the output parser set the format explicitely.

In [38]:
# Inspecting chat_prompt's __repr__
chat_prompt

ChatPromptTemplate(input_variables=['text'], partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='Generate a list of 5 {text}.\n\n{format_instructions}'))])

In [39]:
# Inspecting chain's __repr__
chain

ChatPromptTemplate(input_variables=['text'], partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='Generate a list of 5 {text}.\n\n{format_instructions}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f445967e4d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f4459681cd0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| CommaSeparatedListOutputParser()